In [1]:
import sys
import os
import utils # Importa utilidades visuales (notebooks/utils.py)
from dotenv import load_dotenv

# 1. Configurar el Path del Proyecto
# Agregamos el directorio padre (deep-agents-from-scratch) al path
# para que Python pueda encontrar el paquete "neuro_agent"
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# 2. Cargar Variables de Entorno (.env)
load_dotenv(os.path.join(project_root, ".env"), override=True)

# 3. Habilitar recarga automática de módulos (útil para desarrollo)
%load_ext autoreload
%autoreload 2

print(f"✅ Project Root added to path: {project_root}")


✅ Project Root added to path: /home/juansebas7ian/deep-agents-from-scratch


In [2]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="LangSmith now uses UUID v7", 
    category=UserWarning,
)

# Deep Agent for Research

## Overview 

<img src="./assets/agent_header.png" width="800" style="display:block; margin-left:0;">

Now, we can put everything we have learned together:

* We will use **TODOs** to keep track of tasks. 
* We will use **files** to store raw tool call results. 
* We will **delegate research tasks to sub-agents** for context isolation. 

## Search Tool 

We'll build a search tool that offloads raw contents to files and returns only a summary to the agent. This is a common pattern for long-running agent trajectories, [as we've seen with Manus](https://manus.im/blog/Context-Engineering-for-AI-Agents-Lessons-from-Building-Manus)! 

### Core Components

1. **Search Execution (`run_tavily_search`)**: Performs the actual web search using Tavily API with configurable parameters for results count and topic filtering.

2. **Content Summarization (`summarize_webpage_content`)**: Uses a lightweight model (GPT-4o-mini) to generate structured summaries of webpage content, producing both a descriptive filename and key learnings summary.

3. **Result Processing (`process_search_results`)**: Fetches full webpage content via HTTP, converts HTML to markdown using `markdownify`, and generates summaries for each result.

4. **Context Offloading (`tavily_search` tool)**: The main tool that:
   - Executes search and processes results
   - Saves full content to files in agent state (context offloading)
   - Returns only minimal summaries to the agent (prevents context spam)
   - Uses LangGraph `Command` to update both files and messages

5. **Strategic Thinking (`think_tool`)**: Provides a structured reflection mechanism for agents to analyze findings, assess gaps, and plan next steps in their research workflow.

This architecture solves the token efficiency problem by storing detailed search results in files while keeping the agent's working context minimal and focused.

<div style="background-color: #fff3b0; padding: 10px; border-radius: 4px;">
<b>Note:</b>  
The <code>create_react_agent</code> was moved from the LangGraph library to the LangChain library and renamed to <code>create_agent</code> in the 1.0 code release post-filming. There are slight changes to the imports and code to accommodate this. The video may display the previous configuration.
</div>

In [3]:
%%writefile ../src/deep_agents_from_scratch/research_tools.py



%%writefile ../neuro_agent/src/tools.py
"""Research Tools for NeuroAgent.

This module adapts the original notebook tools to the NeuroAgent architecture,
allowing full compatibility with Notebook 4's research workflow.
"""
import os
from datetime import datetime
import uuid, base64

import httpx
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import InjectedToolArg, InjectedToolCallId, tool
from langgraph.prebuilt import InjectedState
from langgraph.types import Command
from markdownify import markdownify
from pydantic import BaseModel, Field
from tavily import TavilyClient
from typing_extensions import Annotated, Literal

# --- AWS Bedrock Integration ---
from langchain_aws import ChatBedrockConverse
import langchain_aws.chat_models.bedrock_converse as bc

# Configuración de Modelos (Igual que NB4)
llm_nova_lite = ChatBedrockConverse(model="us.amazon.nova-pro-v1:0", region_name="us-east-1", temperature=0.0)
summarization_model = llm_nova_lite

# IMPORTANTE: Usamos el estado de NeuroAgent (que ahora soporta 'files')
from neuro_agent.src.state import AgentState 

tavily_client = TavilyClient()

# --- Prompts ---
SUMMARIZE_WEB_SEARCH = """
You are a research assistant. Summarize the following webpage content.
Date: {date}
Content:
{webpage_content}
"""

class Summary(BaseModel):
    """Schema for webpage content summarization."""
    filename: str = Field(description="Name of the file to store.")
    summary: str = Field(description="Key learnings from the webpage.")

def get_today_str() -> str:
    """Get current date in a human-readable format."""
    return datetime.now().strftime("%a %b %-d, %Y")

def run_tavily_search(
    search_query: str, 
    max_results: int = 1, 
    topic: Literal["general", "news", "finance"] = "general", 
    include_raw_content: bool = True, 
) -> dict:
    """Perform search using Tavily API for a single query."""
    result = tavily_client.search(
        search_query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic
    )
    return result

def summarize_webpage_content(webpage_content: str) -> Summary:
    """Summarize webpage content using the configured summarization model."""
    try:
        structured_model = summarization_model.with_structured_output(Summary)
        summary_and_filename = structured_model.invoke([
            HumanMessage(content=SUMMARIZE_WEB_SEARCH.format(
                webpage_content=webpage_content[:15000], 
                date=get_today_str()
            ))
        ])
        return summary_and_filename
    except Exception:
        return Summary(
            filename="search_result.md",
            summary=webpage_content[:1000] + "..." if len(webpage_content) > 1000 else webpage_content
        )


def process_search_results(results: dict) -> list[dict]:
    """Process search results by summarizing content where available."""
    processed_results = []
    HTTPX_CLIENT = httpx.Client(timeout=30.0)

    for result in results.get('results', []):
        url = result['url']
        try:
            if not result.get('raw_content'):
                response = HTTPX_CLIENT.get(url)
                if response.status_code == 200:
                    raw_content = markdownify(response.text)
                else:
                    raw_content = result.get('content', '')
            else:
                 raw_content = result.get('raw_content', '')
            
            summary_obj = summarize_webpage_content(raw_content)

        except (httpx.TimeoutException, httpx.RequestError, Exception):
            raw_content = result.get('content', '')
            summary_obj = Summary(
                filename="error.md",
                summary="Error processing content."
            )

        uid = base64.urlsafe_b64encode(uuid.uuid4().bytes).rstrip(b"=").decode("ascii")[:8]
        name, ext = os.path.splitext(summary_obj.filename)
        summary_obj.filename = f"{name}_{uid}{ext}"

        processed_results.append({
            'url': result['url'],
            'title': result['title'],
            'summary': summary_obj.summary,
            'filename': summary_obj.filename,
            'raw_content': raw_content,
        })

    return processed_results


@tool
def tavily_search(
    query: str,
    state: Annotated[AgentState, InjectedState], 
    tool_call_id: Annotated[str, InjectedToolCallId],
    max_results: Annotated[int, InjectedToolArg] = 1,
    topic: Annotated[Literal["general", "news", "finance"], InjectedToolArg] = "general",
) -> Command:
    """Search web and save detailed results to files while returning minimal context."""
    search_results = run_tavily_search(
        query,
        max_results=max_results,
        topic=topic,
        include_raw_content=True,
    ) 

    processed_results = process_search_results(search_results)
    
    files = state.get("files", {})
    if files is None: files = {}
    
    saved_files = []
    summaries = []
    
    for i, result in enumerate(processed_results):
        filename = result['filename']
        
        file_content = f"""# Search Result: {result['title']}

**URL:** {result['url']}
**Query:** {query}
**Date:** {get_today_str()}

## Summary
{result['summary']}

## Raw Content
{result['raw_content'] if result['raw_content'] else 'No raw content available'}
"""
        
        files[filename] = file_content
        saved_files.append(filename)
        summaries.append(f"- {filename}: {result['summary']}...")
    
    summary_text = f"""🔍 Found {len(processed_results)} result(s) for '{query}':

{chr(10).join(summaries)}

Files: {', '.join(saved_files)}
💡 Use read_file() to access full details when needed."""

    return Command(
        update={
            "files": files,
            "messages": [
                ToolMessage(summary_text, tool_call_id=tool_call_id)
            ],
        }
    )

@tool
def think_tool(reflection: str) -> str:
    """Tool for strategic reflection on research progress and decision-making."""
    return f"Reflection recorded: {reflection}"

Overwriting ../src/deep_agents_from_scratch/research_tools.py


## Deep Agent

Now, we can just apply all of our prior learnings: 

* We'll give the researcher a `think_tool` and our `search_tool` above.
* We'll give our parent agent file tools, a `think_tool`, and a `task` tool. 

In [4]:
# =============================================================================
# CELDA: Creación del Agente Custom (NeuroAgent Factory) - Control Total
# =============================================================================

from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import SystemMessage

# Importamos el esquema ESTRICTO que definimos previamente.
# Esto garantiza que 'files' y 'todos' tengan sus reducers aplicados correctamente.
from neuro_agent.src.state import AgentState 
try:
    from neuro_agent.src.dynamo_checkpointer import DynamoDBSaver
except ImportError:
    DynamoDBSaver = None

def create_neuro_agent(model, tools, system_prompt, checkpointer):
    """
    Factory function para crear un Agente ReAct con control total sobre el Schema.
    Reemplaza a langgraph.prebuilt.create_react_agent para soportar AgentState custom.
    """
    
    # 1. Componentes del Grafo
    # Aquí es donde ocurre la magia: StateGraph sabe cómo mergear 'files' y 'todos'
    workflow = StateGraph(AgentState)
    model_with_tools = model.bind_tools(tools)
    tool_node = ToolNode(tools)

    # 2. Nodo Agente (Razonamiento)
    def agent_node(state: AgentState):
        messages = state["messages"]
        
        # Inyección dinámica del System Prompt (Runtime)
        if system_prompt:
            # Prepend robusto creando nuevo mensaje
            msg = SystemMessage(content=system_prompt)
            messages = [msg] + messages
            
        # Invocación
        response = model_with_tools.invoke(messages)
        
        # Retornamos solo lo nuevo. LangGraph usará el reducer 'add_messages' 
        return {"messages": [response]}

    # 3. Definición de Nodos y Aristas
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", tool_node)

    workflow.add_edge(START, "agent")

    # Lógica Condicional: ¿El LLM quiere usar una herramienta?
    def should_continue(state: AgentState):
        last_message = state["messages"][-1]
        if last_message.tool_calls:
            return "tools"
        return END

    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "tools": "tools", # Si hay tool_calls, ir a tools
            END: END          # Si no, terminar y responder al usuario
        }
    )

    workflow.add_edge("tools", "agent") # Loop ReAct: Tool -> Agent

    # 4. Compilación con Persistencia
    return workflow.compile(checkpointer=checkpointer)

# --- CONFIGURACIÓN DE CHECKPOINTER (DynamoDB) ---
try:
    if DynamoDBSaver:
        checkpointer = DynamoDBSaver(table_name="LangGraphCheckpoints")
        print(f"✅ DynamoDB Checkpointer Initialized (Table: LangGraphCheckpoints)")
    else:
        raise ImportError("DynamoDBSaver not found")
except Exception as e:
    print(f"❌ DynamoDB Error: {e}")
    # Fallback seguro
    from langgraph.checkpoint.memory import MemorySaver
    checkpointer = MemorySaver()



✅ DynamoDB Checkpointer Initialized (Table: LangGraphCheckpoints)


In [10]:
from datetime import datetime
from IPython.display import Image, display

# --- NeuroAgent Core ---
from neuro_agent.src.state import AgentState
from langgraph.graph import StateGraph, START, END
# Checkpoints: Usar MemorySaver (RAM) o AsyncRedisSaver (Redis)
from langgraph.checkpoint.memory import MemorySaver 
try:
    from langgraph.checkpoint.redis.aio import AsyncRedisSaver
except ImportError:
    AsyncRedisSaver = None # Manejo seguro si no está instalado

from langgraph.prebuilt import ToolNode, create_react_agent
from langchain_core.tools import tool

# --- Tools ---
from neuro_agent.src.tools import (
    tavily_search, think_tool, ls, read_file, write_file, 
    write_todos, read_todos, get_today_str
)

# --- Prompts ---
from deep_agents_from_scratch.prompts import (
    RESEARCHER_INSTRUCTIONS,
    SUBAGENT_USAGE_INSTRUCTIONS
)

# # --- Models ---
# from langchain_aws import ChatBedrockConverse
# llm_nova_lite = ChatBedrockConverse(model="us.amazon.nova-lite-v1:0", region_name="us-east-1", temperature=0.0)
# model = llm_nova_lite

# # --- 1. Definir Sub-Agente ---
# sub_agent_tools = [tavily_search, think_tool]
# research_sub_agent_config = {
#     "name": "research-agent",
#     "description": "Delegate research to the sub-agent.",
#     "prompt": RESEARCHER_INSTRUCTIONS.format(date=get_today_str()),
# }

# # --- 2. Task Tool ---
# def _create_task_tool(tools, sub_agent_config, llm, state_schema):
#     """Creates a tool that invokes a sub-agent graph."""
#     system_prompt = sub_agent_config["prompt"]
#     sub_graph = create_react_agent(llm, tools=tools, prompt=system_prompt)
    
#     @tool
#     def delegate_research(task: str, context: str = "") -> str:
#         """Delegate a research task to the specialist researcher."""
#         response = sub_graph.invoke({"messages": [("user", f"Task: {task}\nContext: {context}")]})
#         return response["messages"][-1].content
    
#     return delegate_research

# task_tool = _create_task_tool(
#     sub_agent_tools, research_sub_agent_config, model, AgentState
# )

# # --- 3. Herramientas ---
# built_in_tools = [ls, read_file, write_file, write_todos, read_todos, think_tool]
# delegation_tools = [task_tool] 
# all_tools = sub_agent_tools + built_in_tools + delegation_tools

# --- 4. Prompt Principal ---
SUBAGENT_INSTRUCTIONS_TEXT = SUBAGENT_USAGE_INSTRUCTIONS.format(
    max_concurrent_research_units=3,
    max_researcher_iterations=3,
    date=datetime.now().strftime("%a %b %-d, %Y"),
)
SUPERVISOR_PROMPT = f"""You are a Supervisor Agent.
{SUBAGENT_INSTRUCTIONS_TEXT}
Always verify your plan using 'think_tool' before acting.
"""

# # --- 5. Grafo (Checkpointer Selection) ---
# # Opción: Redis Async (Descomentar si tienes Redis)
# # redis_url = "redis://localhost:6379"
# # checkpointer = AsyncRedisSaver.from_conn_string(redis_url) if AsyncRedisSaver else MemorySaver()

# # Default: Memory (Seguro para demos locales)
# checkpointer = MemorySaver()

# app = create_react_agent(
#     model, 
#     tools=all_tools, 
#     checkpointer=checkpointer,
#     prompt=SUPERVISOR_PROMPT
# )

# print(f"✅ Agent Graph Initialized using: {type(checkpointer).__name__}")

In [12]:
# Imports de Utility
from utils import show_prompt, format_messages

show_prompt(RESEARCHER_INSTRUCTIONS)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  You are a research assistant conducting research on the user's input topic. For context, today's date is       │
│  {date}.                                                                                                        │
│                                                                                                                 │
│  <Task>                                                                                                         │
│  Your job is to use tools to gather information about the user's input topic.                                   │
│  You can use any of the tools provided to you to find resources that can help answer the research question.     │
│  You can call these tools in series or in parallel, your research is conducted in a tool-calling loop.          │
│  </Task>                                                                                                        │
│                                                                                                                 │
│  <Available Tools>                                                                                              │
│  You have access to two main tools:                                                                             │
│  1. **tavily_search**: For conducting web searches to gather information                                        │
│  2. **think_tool**: For reflection and strategic planning during research                                       │
│                                                                                                                 │
│  **CRITICAL: Use think_tool after each search to reflect on results and plan next steps**                       │
│  </Available Tools>                                                                                             │
│                                                                                                                 │
│  <Instructions>                                                                                                 │
│  Think like a human researcher with limited time. Follow these steps:                                           │
│                                                                                                                 │
│  1. **Read the question carefully** - What specific information does the user need?                             │
│  2. **Start with broader searches** - Use broad, comprehensive queries first                                    │
│  3. **After each search, pause and assess** - Do I have enough to answer? What's still missing?                 │
│  4. **Execute narrower searches as you gather information** - Fill in the gaps                                  │
│  5. **Stop when you can answer confidently** - Don't keep searching for perfection                              │
│  </Instructions>                                                                                                │
│                                                                                                                 │
│  <Hard Limits>                                                                                                  │
│  **Tool Call Budgets** (Prevent excessive searching):                                                           │
│  - **Simple queries**: Use 1-2 search tool calls maximum                                                        │
│  - **Normal queries**: Use 2-3 search tool calls maximum                                                        │
│  - **Very Complex queries**: Use up to 5 search tool calls maximum                                              │
│  - **Always stop**: After 5 search tool calls if you c

In [14]:
# Imports de Prompts y Utilidades
from deep_agents_from_scratch.prompts import (
    RESEARCHER_INSTRUCTIONS,
    SUBAGENT_USAGE_INSTRUCTIONS,
    TODO_USAGE_INSTRUCTIONS, 
    FILE_USAGE_INSTRUCTIONS  
)
from utils import show_prompt, format_messages

# --- 4. Prompt Principal (Ensamblaje Exacto) ---

# Configuración de variables (si no estaban definidas antes)
max_concurrent_research_units = 3
max_researcher_iterations = 3

# Construir SUBAGENT_INSTRUCTIONS
SUBAGENT_INSTRUCTIONS = SUBAGENT_USAGE_INSTRUCTIONS.format(
    max_concurrent_research_units=max_concurrent_research_units,
    max_researcher_iterations=max_researcher_iterations,
    date=datetime.now().strftime("%a %b %-d, %Y"),
)

# Ensamblar INSTRUCTIONS (Tu código exacto)
INSTRUCTIONS = (
    "# TODO MANAGEMENT\n"
    + TODO_USAGE_INSTRUCTIONS
    + "\n\n"
    + "=" * 80
    + "\n\n"
    + "# FILE SYSTEM USAGE\n"
    + FILE_USAGE_INSTRUCTIONS
    + "\n\n"
    + "=" * 80
    + "\n\n"
    + "# SUB-AGENT DELEGATION\n"
    + SUBAGENT_INSTRUCTIONS
)

# Visualizar
show_prompt(INSTRUCTIONS)


╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  # TODO MANAGEMENT                                                                                              │
│  ⛔ MANDATORY EXECUTION CONTRACT — The TODO list is NOT optional.                                               │
│                                                                                                                 │
│  ## Rules (Non-Negotiable)                                                                                      │
│  1. ALWAYS create a TODO plan using write_todos as your FIRST action on ANY user request.                       │
│  2. EVERY step in the TODO MUST be executed using the appropriate tool — NO exceptions.                         │
│  3. NEVER answer from memory alone — you MUST use tools to complete each step.                                  │
│  4. After executing a step, mark it as completed using write_todos.                                             │
│  5. Use read_todos frequently to verify your progress and remind yourself of remaining steps.                   │
│  6. You CANNOT finish until ALL steps show status "completed".                                                  │
│  7. The system will BLOCK you from ending if steps remain incomplete.                                           │
│  8. If you have already executed tools in this turn, mark the corresponding steps as 'completed' in your        │
│  initial plan.                                                                                                  │
│                                                                                                                 │
│  ## Workflow                                                                                                    │
│  1. Create your TODO plan (write_todos)                                                                         │
│  2. Execute step 1 using the appropriate tool                                                                   │
│  3. Mark step 1 completed (write_todos)                                                                         │
│  4. Read TODOs to check progress (read_todos)                                                                   │
│  5. Repeat for each remaining step                                                                              │
│  6. Only provide your final answer AFTER all TODOs are completed                                                │
│                                                                                                                 │
│  IMPORTANT: Aim to batch research tasks into a *single TODO* in order to minimize the number of TODOs you have  │
│  to keep track of.                                                                                              │
│  IMPORTANT: The system audits your execution. Skipped steps are flagged. Follow the script.                     │
│                                                                                                                 │
│                                                                                                                 │
│  ================================================================================                               │
│                                                                                                                 │
│  # FILE SYSTEM USAGE                                                                                            │
│  You have access to a virtual file system to help you retain and save context.                                  │
│                                                                                                                 │
│  ## ⚠️ ABSOLUTE FIRST STEP - NO EXCEPTIONS ⚠️          

In [15]:
# --- INSTANCIACIÓN ---
print("🏗️ Construyendo NeuroAgent con StateGraph manual...")

# Usamos la función custom que acabamos de definir
agent = create_neuro_agent(
    model=model, 
    tools=all_tools, 
    system_prompt=INSTRUCTIONS, 
    checkpointer=checkpointer 
)

print("✅ NeuroAgent creado exitosamente. El esquema AgentState está activo.")

# Visualizar
try:
    display(Image(agent.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    print("No se pudo generar la imagen del grafo (requiere mermaid/graphviz).")

🏗️ Construyendo NeuroAgent con StateGraph manual...


NameError: name 'model' is not defined

In [ ]:
# Dedup tools by name, keeping order
seen = set()
unique_tools = []
for tool in all_tools:
    if tool.name not in seen:
        unique_tools.append(tool)
        seen.add(tool.name)

all_tools = unique_tools
print(f"✅ Tools deduplicated: {[t.name for t in all_tools]}")

# Re-crear el agente con la lista limpia
agent = create_neuro_agent(
    model, 
    all_tools, 
    system_prompt=INSTRUCTIONS, 
    checkpointer=checkpointer
)
print("✅ Agent Re-Created properly.")

✅ Tools deduplicated: ['tavily_search', 'think_tool', 'ls', 'read_file', 'write_file', 'write_todos', 'read_todos', 'delegate_research']
✅ Agent Re-Created properly.


In [31]:
# 1. Deduplicar Tools (Para arreglar ValidationException)
seen = set()
unique_tools = []
for tool in all_tools:
    if tool.name not in seen:
        unique_tools.append(tool)
        seen.add(tool.name)

all_tools = unique_tools
print(f"✅ Tools deduplicated: {[t.name for t in all_tools]}")

# 2. Re-crear Agente (Para aplicar el cambio)
app = create_neuro_agent(
    model, 
    all_tools, 
    system_prompt=INSTRUCTIONS, 
    checkpointer=checkpointer
)
agent = app

print("✅ Agent Re-Created. Ready to Run.")

✅ Tools deduplicated: ['tavily_search', 'think_tool', 'ls', 'read_file', 'write_file', 'write_todos', 'read_todos', 'delegate_research']
✅ Agent Re-Created. Ready to Run.


In [32]:
from utils import format_messages
import uuid

# Generamos un ID nuevo (o usa uno existente para probar memoria a largo plazo)
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

print(f"🆔 Running with Thread ID: {thread_id}")

# Invocamos al Agente (Usando DynamoDB/MemoryCheckpointer implícitamente via config)
result = agent.invoke(
    {
        "messages": [
            ("user", "Give me an overview of Model Context Protocol (MCP).")
        ],
    },
    config=config
)

# Mostramos el resultado formateado
# Nota: result["messages"] contiene todo el historial (User + AI)
format_messages(result["messages"])

🆔 Running with Thread ID: 8821403c-acb9-4bf3-953d-9f088e83bf04


NotImplementedError: 

Trace: 
https://smith.langchain.com/public/3a389ec6-8e6e-4f9e-9a82-0d0a9569e6f8/r
<!-- https://smith.langchain.com/public/1df7a10e-1465-499c-a3e0-86c1d5429324/r -->

## Using the Deep Agent Package

Now you understand the underlying patterns! 

You can [use the `deepagents` package](
https://github.com/hwchase17/deepagents) as a simple abstraction:

* It include the file system tools
* It includes the todo tool
* It includes the task tool

You only need to supply the sub-agent and any tools you want the sub-agent to use.

In [9]:
from deepagents import create_deep_agent
# updated since filming

# Create research sub-agent
research_sub_agent_2 = {
    "name": "research-agent",
    "description": "Delegate research to the sub-agent researcher. Only give this researcher one topic at a time.",
    "system_prompt": RESEARCHER_INSTRUCTIONS.format(date=get_today_str()),
    "tools": [tavily_search, think_tool],
}


agent = create_deep_agent(  # updated 
    tools=sub_agent_tools,
    system_prompt=INSTRUCTIONS,
    subagents=[research_sub_agent_2],
    model=model,
)

# Show the agent
#display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [10]:
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Give me an very brief overview of Model Context Protocol (MCP).",
            }
        ],
    }
)

format_messages(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Give me an very brief overview of Model Context Protocol (MCP).                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: ls                                                                                                │
│    Args: {                                                                                                      │
│   "path": "/"                                                                                                   │
│ }                                                                                                               │
│    ID: tooluse_WwcXS4WjqVdSD7upqc8x43                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: write_file                                                                                        │
│    Args: {                                                                                                      │
│   "file_path": "/user_request.md",                                                                              │
│   "content": "Give me an very brief overview of Model Context Protocol (MCP)."                                  │
│ }                                                                                                               │
│    ID: tooluse_sFS5ttHj9Racle0wZFEM3E                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Updated file /user_request.md                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: task                                                                                              │
│    Args: {                                                                                                      │
│   "description": "Research and provide a very brief overview of Model Context Protocol (MCP)",                  │
│   "subagent_type": "research-agent"                                                                             │
│ }                                                                                                               │
│    ID: tooluse_bF5nXk74DScMjyeDdXaXRC                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Based on my existing knowledge, here's a very brief overview of Model Context Protocol (MCP):                   │
│                                                                                                                 │
│ **Model Context Protocol (MCP)** is a communication framework designed to enable large language models (LLMs)   │
│ to interact with external systems, data sources, and tools. It provides a structured way for LLMs to:           │
│                                                                                                                 │
│ 1. Request information from external sources                                                                    │
│ 2. Execute specific functions or tools                                                                          │
│ 3. Process and receive responses                                                                                │
│                                                                                                                 │
│ This protocol helps bridge the gap between isolated LLMs and the broader software ecosystem, allowing for more  │
│ dynamic and context-aware AI interactions where models can access real-time data and functionality beyond their │
│ trained knowledge.                                                                                              │
│                                                                                                                 │
│ **Important Note:** This overview is based on pre-existing knowledge due to technical limitations preventing    │
│ current web research verification. For the most up-to-date and detailed information, consulting official MCP    │
│ documentation or recent technical papers would be recommended.                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: write_todos                                                                                       │
│    Args: {                                                                                                      │
│   "todos": [                                                                                                    │
│     {                                                                                                           │
│       "content": "Provide brief overview of Model Context Protocol (MCP) based on research",                    │
│       "status": "completed"                                                                                     │
│     }                                                                                                           │
│   ]                                                                                                             │
│ }                                                                                                               │
│    ID: tooluse_lLTLkGbMOOqnSHywH5DGhA                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Updated todo list to [{'content': 'Provide brief overview of Model Context Protocol (MCP) based on research',   │
│ 'status': 'completed'}]                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Here's a very brief overview of Model Context Protocol (MCP):                                                   │
│                                                                                                                 │
│ **Model Context Protocol (MCP)** is a communication framework designed to enable large language models (LLMs)   │
│ to interact with external systems, data sources, and tools. It provides a structured way for LLMs to:           │
│                                                                                                                 │
│ 1. Request information from external sources                                                                    │
│ 2. Execute specific functions or tools                                                                          │
│ 3. Process and receive responses                                                                                │
│                                                                                                                 │
│ This protocol helps bridge the gap between isolated LLMs and the broader software ecosystem, allowing for more  │
│ dynamic and context-aware AI interactions where models can access real-time data and functionality beyond their │
│ trained knowledge.                                                                                              │
│                                                                                                                 │
│ *Note: This overview is based on pre-existing knowledge. For the most current and detailed information,         │
│ consulting official MCP documentation would be recommended.*                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Trace: 
https://smith.langchain.com/public/1d626d81-a102-4588-a2fb-cab40a7271f1/r
<!-- https://smith.langchain.com/public/1ae2d7f6-f901-4ebd-b6c3-6657a55f88ae/r -->